## Monte Carlo Tree Search
Monte Carlo Tree Search is the industry standard for complex games like Go and Chess. Combined with ConvNets, MCTS was used by Google in Alpha Go and Alpha Zero. In MCTS, the computer plays a number of games against itself. It generates its own expected value of each game state through trial and error, which eliminates the need to explore every state. The cost of this is non-optimality. As the computer simulates more games, it will learn which game states are favorable or unfavorable (more likely to result in a victory). In this section, you will implement Monte Carlo Tree Search for tic tac toe.

### Introduction

We will be reusing some tic tac toe code from the previous lab, namely get_possible_moves and get_score


In [ ]:
# Lab by Morgan Swanson
import numpy as np
from tqdm.notebook import tqdm
example_board = np.array([[' ', ' ', ' '],
                          [' ', ' ', ' '],
                          [' ', ' ', ' ']])

# Calculates all successor states to a given state
def get_possible_moves(board, player):
    moves = []
    for (x, y), element in np.ndenumerate(board):
        if element == ' ':
            new_board = np.array(board, copy=True)
            new_board[x][y] = 'X' if player is 'max' else 'O'
            moves.append(new_board)
    return moves


def get_score(board, depth=0):
    if (np.any(np.all(board == 'X', axis=0)) or
        np.any(np.all(board == 'X', axis=1)) or
        np.all(board.diagonal() == 'X') or
        np.all(np.fliplr(board).diagonal() == 'X')):
        # Max Victory
        return 1 * (1 / (1 + depth))
    elif (np.any(np.all(board == 'O', axis=0)) or
          np.any(np.all(board == 'O', axis=1)) or
          np.all(board.diagonal() == 'O') or
          np.all(np.fliplr(board).diagonal() == 'O')):
        # Min Victory
        return -1 * (1 / (1 + depth))
    elif not (board == ' ').any():
        # Draw
        return 0
    else:
        # Unfinished Game
        return None

### Storing Simluation Results

In order to remember how good each state is, we will keep track of the results of a state in a class called Metrics. This class will be mapped to the hash of each board state (first converted to a string, since keys must be immutable in python), so that they can easily be looked up given a board configuration. Each board state will have its own Metrics Class.



In [ ]:
import math
class Metrics:

    def __init__(self):
        self.min_wins = 0
        self.max_wins = 0
        self.count = 0

    def update(self, score):
        if score > 0:
            self.max_wins += 1
        elif score < 0:
            self.min_wins += 1
        self.count += 1

    def get_p_win(self, player):
        try:
            if player == 'min':
                return self.min_wins / self.count
            elif player == 'max':
                return self.max_wins / self.count
            else:
                raise ValueError('player {} must be min or max'.format(player))
        except ZeroDivisionError:
            raise ValueError('must be updated at least once \
                              to get win probability')

    def get_expected_value(self, player):
        try:
            if player == 'min':
                return (self.min_wins - self.max_wins) / self.count
            elif player == 'max':
                return (self.max_wins -self.min_wins) / self.count
            else:
                raise ValueError('player {} must be min or max'.format(player))
        except ZeroDivisionError:
            raise ValueError('must be updated at least once \
                              to get expected value')

    def get_explore_term(self, parent, c=1.41):
        if parent.count:
            return c * (math.log(parent.count) / self.count) ** (1 / 2)
        else:
            return 0


    def get_ucb(self, player, parent, c=1.41, default=6):
        if self.count:
            p_win = self.get_p_win(player)
            explore_term = self.get_explore_term(parent)
            return p_win + explore_term
        else:
            return default

### History Table
To keep track of these metrics, we will store them an a table which we call a 'history'. This table is indexed by the string of the board. The following example shows how a history is used to update a Metric for a given board.

In [ ]:
# Create python dictionary for results
example_history = {}
# Create a new Metrics entry for the example board
example_history[example_board.tostring()] = Metrics()
print(example_history)
# Tell the metrics class that the simluation resulted in a loss
print("Simulate One Loss")
example_history[example_board.tostring()].update(-1)
print("Win Percentage:",
      example_history[example_board.tostring()].get_p_win("max"))
# Tell the metrics class that two simulations resulted in wins
print("Simulate Two Wins")
example_history[example_board.tostring()].update(1)
example_history[example_board.tostring()].update(1)
print("Win Percentage:",
      example_history[example_board.tostring()].get_p_win("max"))

{b' \x00\x00\x00 \x00\x00\x00 \x00\x00\x00 \x00\x00\x00 \x00\x00\x00 \x00\x00\x00 \x00\x00\x00 \x00\x00\x00 \x00\x00\x00': <__main__.Metrics object at 0x7f0e8bad6290>}
Simulate One Loss
Win Percentage: 0.0
Simulate Two Wins
Win Percentage: 0.6666666666666666


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  after removing the cwd from sys.path.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  # Remove the CWD from sys.path while we load stuff.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  del sys.path[0]
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:16: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  app.launch_new_instance()


***
### Upper Confidence Bound
In order to decide which game path to simulate, Monte Carlo Tree search calculates the Upper Confidence Bound to heuristically determine which move to try next.  This metric balances deep exploitation of promising moves with exploration of unknown options. It will choose the move in which $\frac{w_i}{n_i} + c\sqrt{\frac{\ln N_i}{n_i}}$ has the highest value. In this formula:
* $w_i$ stands for the number of wins for the node considered after the $i$-th move
* $n_i$ stands for the number of simulations for the node considered after the $i$-th move
* $N_i$ stands for the total number of simulations after the $i$-th move ran by the parent node of the one considered
* $c$ is the exploration parameter—theoretically equal to $\sqrt2$; in practice usually chosen empirically

Notice how if $n_i$ is 0, this formula is undefined. Therefore, I set a default value of an unexplored node to 6 to ensure each node is explored at least once. Notice that the UCB requires the Metrics from the previous move made (the parent node).
We will calcuate the Upper Confidence Bound as shown:

In [ ]:
# Creating a possible move
example_child = get_possible_moves(example_board, "max")[0]
print('Example Child\n', example_child)
# Initializing the Metrics class for this board
example_history[example_child.tostring()] = Metrics()
print("UCB:", example_history[example_child.tostring()]
                .get_ucb("max", example_history[example_board.tostring()]))
# Let's say this simulation resulted in a win
print("Simulate One Win")
example_history[example_child.tostring()].update(1)
print("UCB:", example_history[example_child.tostring()]
                .get_ucb("max", example_history[example_board.tostring()]))


Example Child
 [['X' ' ' ' ']
 [' ' ' ' ' ']
 [' ' ' ' ' ']]
UCB: 6
Simulate One Win
UCB: 2.477887374295169


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  """
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  import sys
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  # Remove the CWD from sys.path while we load stuff.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  # This is added back by InteractiveShellApp.init_path()
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  if sys.path[0] == '':


### Board Selection
Your first task is to implement board selection. Within a simulation, we must be able to, given a board, select a successor board. We will do this by calculating the UCB for all possible successors and choosing randomly between the boards that have the highest UCB.

Formally, when given the current board, the table of metrics for all known boards, and the current player, select_board will return a sucessor board. Right now, select board returns a random successor. The model implementation is 13 lines long.

In [ ]:
# Select the next board to simulate using upper confidence bound
def select_board(board, history, player):
    boards = get_possible_moves(board, player)
    ucb = np.array([])
    for table in boards:
      if table.tostring() not in history:
        history[table.tostring()] = Metrics()

      ucb = np.append(ucb, history[table.tostring()].get_ucb(player, history[board.tostring()]))
    winner = np.argwhere(ucb==np.amax(ucb))
    return boards[np.random.choice(winner.flatten())]


select_board(example_board, example_history, 'max')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  if __name__ == '__main__':
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  import sys


array([[' ', 'X', ' '],
       [' ', ' ', ' '],
       [' ', ' ', ' ']], dtype='<U1')

### Running the simulations

Your second task is to implement the simulations. The simulations will update the Metrics of the board states within the history table. We will run N simulations, using the UCB combined with random selection to generate a path through the game tree. You must start with the given board and select successor boards using 'select_board' until a terminal state is reached and then update the metrics along that game path with the result.

In [ ]:
# Run a batch of monte carlo simulations, defaults to 100 simulations
def run_simulations(board, history, player, count=200):
    current = board
    path = []
    for i in tqdm(range(count)):
        # Adding some random data to the succesor board states
        # This code should be replaced by your simulation code
        #score = get_score(current)
        while get_score(current) is None:
          path.append(current)
          if current.tostring() not in history:
            history[current.tostring()] = Metrics()
          if (len(path) - 1) % 2 == 0:
            current = select_board(current, history, 'max')
          else:
            current = select_board(current, history, 'min')
        path.append(current)
        if current.tostring() not in history:
          history[current.tostring()] = Metrics()
        for node in path:
          history[node.tostring()].update(get_score(current))
        path = []
        current = board


### Selecting the Best Move

After N simulations, we must eventually make a decision of the best move using the expected value of each possible move.

In [ ]:
# Finds the best move (state with highest/lowest value)
def find_best_move(board, history, player):
    if board.tostring() not in history:
        history[board.tostring()] = Metrics()
    print("Deciding best move...")
    run_simulations(board, history, player, count=60)
    boards = get_possible_moves(board, player)
    print(history[b.tostring()].count for b in boards)
    values = [history[b.tostring()].get_expected_value(player) for b in boards]
    return boards[np.argmax(values)]


# Testing find best move
find_best_move(example_board, example_history, 'max')

Deciding best move...


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  This is separate from the ipykernel package so we can avoid doing imports until


  0%|          | 0/60 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  # This is added back by InteractiveShellApp.init_path()
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  if __name__ == '__main__':
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  import sys
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:18: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:21: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.


<generator object find_best_move.<locals>.<genexpr> at 0x7f0e8bacd450>


array([[' ', ' ', ' '],
       [' ', 'X', ' '],
       [' ', ' ', ' ']], dtype='<U1')

### Demo


In [ ]:
# Starts a game against the AI Program
def run_demo():
    board = np.array([[' ', ' ', ' '],
                      [' ', ' ', ' '],
                      [' ', ' ', ' ']])
    history = {}
    score = get_score(board)
    player = "max"
    while score is None:
        if player == "max":
            board = find_best_move(board, history, player)
        else:
            move_entered = False
            while not move_entered:
                try:
                    move = int(input('Choose a move...')) - 1
                    if not 0 <= move <= 8:
                        print("Enter an integer between 1 and 9.\n")
                        continue
                    elif not board[move//3][move%3] == ' ':
                        print("That spot is already taken.\n")
                        continue
                    else:
                        board[move//3][move%3]= 'O'
                        move_entered = True
                except ValueError:
                    print("Enter an integer.\n")
        score = get_score(board)
        player = "min" if player == "max" else "max"
        print(board)
    if (score == 0):
        print("Draw")
    elif (score > 0):
        print("You Lose")
    else:
        print("You Win")


run_demo()

Deciding best move...


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  after removing the cwd from sys.path.


  0%|          | 0/60 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  # This is added back by InteractiveShellApp.init_path()
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  import sys
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  if __name__ == '__main__':
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:18: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:21: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.


<generator object find_best_move.<locals>.<genexpr> at 0x7f0e8b81b250>
[['X' ' ' ' ']
 [' ' ' ' ' ']
 [' ' ' ' ' ']]
Choose a move...5
[['X' ' ' ' ']
 [' ' 'O' ' ']
 [' ' ' ' ' ']]
Deciding best move...


...

<generator object find_best_move.<locals>.<genexpr> at 0x7f0e8b81b850>
[['X' ' ' ' ']
 ['O' 'O' 'X']
 ['X' ' ' ' ']]
Choose a move...3
[['X' ' ' 'O']
 ['O' 'O' 'X']
 ['X' ' ' ' ']]
Deciding best move...


  0%|          | 0/60 [00:00<?, ?it/s]

<generator object find_best_move.<locals>.<genexpr> at 0x7f0e8b81ba50>
[['X' ' ' 'O']
 ['O' 'O' 'X']
 ['X' 'X' ' ']]
Choose a move...9
[['X' ' ' 'O']
 ['O' 'O' 'X']
 ['X' 'X' 'O']]
Deciding best move...


  0%|          | 0/60 [00:00<?, ?it/s]

<generator object find_best_move.<locals>.<genexpr> at 0x7f0e8b81bc50>
[['X' 'X' 'O']
 ['O' 'O' 'X']
 ['X' 'X' 'O']]
Draw


In [ ]:
5


5